In [1]:
import scanpy as sc
import numpy as np 
import pandas as pd
# openpyxl package to read excel# 'pip install openpyxl' if undownload it

In [11]:
supplementaryTable = "Supplementary table.xlsx"
dataset = "PBMC_CITE"
dataset_url = "https://atlas.fredhutch.org/data/nygc/multimodal/pbmc_multimodal.h5seurat"
CellType_ColumnName = 'celltype.l2'

In [3]:
# dataset download
from os.path import isfile
dataset_download_path = dataset_url.split('/')[-1]
if not isfile(dataset_download_path):
    !wget -O {dataset_download_path} {dataset_url}

--2022-02-28 17:22:59--  https://atlas.fredhutch.org/data/nygc/multimodal/pbmc_multimodal.h5seurat
Resolving atlas.fredhutch.org (atlas.fredhutch.org)... 13.35.7.62, 13.35.7.114, 13.35.7.28, ...
Connecting to atlas.fredhutch.org (atlas.fredhutch.org)|13.35.7.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2213622763 (2.1G) [application/x-www-form-urlencoded]
Saving to: ‘pbmc_multimodal.h5seurat’

pbmc_multimodal.h5s 100%[===================>]   2.06G  40.8MB/s    in 45s     

2022-02-28 17:23:46 (46.4 MB/s) - ‘pbmc_multimodal.h5seurat’ saved [2213622763/2213622763]



In [6]:
def read_h5seurat(filename):
    import h5py
    import scanpy as sc
    from scipy import sparse
    import pandas as pd
    ds = h5py.File(filename, 'r')
    m=ds['assays/SCT/counts']
    def get_loop(dir):
        re = {}
        for k in ds[dir].keys():
            if k!='_index':
                re[k] = ds[dir][k][:].astype(str)
        return re 
    def get_once(dir):
        return ds[dir][:].astype(str)
    ad=sc.AnnData(
      X = sparse.csr_matrix((m['data'][:],m['indices'][:],m['indptr'][:])),
      obs = pd.DataFrame(get_loop('meta.data'),index=get_once('meta.data/_index')),
      var = pd.DataFrame(get_once('assays/SCT/features'),columns=['Gene'])
    )
    return ad

In [15]:
adata = read_h5seurat(dataset_download_path)
adata.X = adata.X.tocsr()
df = pd.read_excel(supplementaryTable, sheet_name=dataset)

In [16]:
def add_celltype(adata,celltype_columnname):
    adata.obs[celltype_columnname] = adata.obs[celltype_columnname].astype("category")
    Transferlist = [np.where(df['origin cell type']==CellTypeList)[0][0] for CellTypeList in adata.obs[celltype_columnname].values.categories]
    adata.obs['modelA id'] = df['modelA id'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
    adata.obs['modelC id'] = df['modelC id'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
    adata.obs['transfer cell type'] = df['transfer cell type'][Transferlist].values[adata.obs[celltype_columnname].values.codes]
add_celltype(adata,CellType_ColumnName)

In [ ]:
adata.obs['origin cell type'] = adata.obs['celltype.l2']

In [17]:
adata.obs

,Phase,celltype.l1,celltype.l2,celltype.l3,donor,lane,nCount_ADT,nCount_RNA,nCount_SCT,nFeature_ADT,nFeature_RNA,nFeature_SCT,orig.ident,time,modelA id,modelC id,transfer cell type
L1_AAACCCAAGAAACTCA,G1,Mono,CD14 Mono,CD14 Mono,P2,L1,7430.0,10823.0,6380.0,221,2915,2548,P2_7,7,10,8,monocyte
L1_AAACCCAAGACATACA,G1,CD4 T,CD4 TCM,CD4 TCM_1,P1,L1,5949.0,5864.0,5693.0,211,1617,1615,P1_7,7,0,0,T-helper cell
L1_AAACCCACAACTGGTT,S,CD8 T,CD8 Naive,CD8 Naive,P4,L1,6547.0,5067.0,5066.0,217,1381,1379,P4_3,3,1,1,cytotoxic T cell
L1_AAACCCACACGTACTA,G1,NK,NK,NK_2,P3,L1,3508.0,4786.0,4984.0,207,1890,1889,P3_7,7,7,5,natural killer cell
L1_AAACCCACAGCATACT,G1,CD8 T,CD8 Naive,CD8 Naive,P4,L1,6318.0,6505.0,5854.0,219,1621,1620,P4_7,7,1,1,cytotoxic T cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,S,CD8 T,CD8 Naive,CD8 Naive,P5,E2L8,4089.0,9346.0,8231.0,200,2201,2200,P5_7,7,1,1,cytotoxic T cell
E2L8_TTTGTTGGTGTGCCTG,G1,Mono,CD14 Mono,CD14 Mono,P5,E2L8,6869.0,9318.0,8680.0,213,2938,2938,P5_3,3,10,8,monocyte
E2L8_TTTGTTGGTTAGTTCG,S,B,B intermediate,B intermediate kappa,P8,E2L8,4173.0,11619.0,8555.0,208,3224,3130,P8_0,0,2,2,memory B cell
E2L8_TTTGTTGGTTGGCTAT,G1,Mono,CD16 Mono,CD16 Mono,P5,E2L8,5979.0,15436.0,8970.0,221,3999,3533,P5_3,3,10,8,monocyte


In [18]:
adata.write_h5ad(f'../{dataset}.h5ad')

/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Phase' as categorical
/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'celltype.l1' as categorical
/home/shhsu/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused c

In [19]:
!rm {dataset_download_path}